In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread, imsave
import cv2
import pandas as pd
import seaborn as sns
import os
from os import listdir 
from sklearn.metrics import jaccard_score
 
from freatures_funciones import *

from skimage import filters as skifilters
from scipy import ndimage
import skimage
from skimage import filters

## Extración de características

### Funciones

Antes de definir la función la cual extraerá las caracteristicas de cada imagen, necesitamos definir funciones auxiliares. La primera es *grey_edge*, la cual estima la iluminación de la imagen.

In [2]:
def grey_edge(image, njet=0, mink_norm=1, sigma=1):
    """
    Estima el la fuente de iluminación tal como se propone en:
    J. van de Weijer, Th. Gevers, A. Gijsenij
    "Edge-Based Color Constancy"
    IEEE Trans. Image Processing, accepted 2007.
    Dependiendo de los parametros, la estimación es igual a Grey-World, Max-RGB, general Grey-World,
    Shades-of-Grey o Grey-Edge algorithm.

    :param image: imagen rgb (NxMx3)
    :param njet: orden de diferenciación (range from 0-2)
    :param mink_norm: norma minkowski norm usada (si mink_norm==-1 entonces se aplica
     la operacion max, que equivale a tomar minkowski_norm=infinity).
    :param sigma: sigma usado para el filtro gaussiano
    :return: estimación de la fuente de iluminación
    :raise: ValueError
    
    Ref: https://github.com/MinaSGorgy/Color-Constancy
    """
    gauss_image = filters.gaussian(image, sigma=sigma, channel_axis=True)

    if njet == 0:
        deriv_image = [gauss_image[:, :, channel] for channel in range(3)]
    else:   
        if njet == 1:
            deriv_filter = filters.sobel
        elif njet == 2:
            deriv_filter = filters.laplace
        else:
            raise ValueError("njet should be in range[0-2]! Given value is: " + str(njet))     
        deriv_image = [np.abs(deriv_filter(gauss_image[:, :, channel])) for channel in range(3)]

    for channel in range(3):
        deriv_image[channel][image[:, :, channel] >= 255] = 0.

    if mink_norm == -1:  
        estimating_func = np.max 
    else:
        estimating_func = lambda x: np.power(np.sum(np.power(x, mink_norm)), 1 / mink_norm)
    illum = [estimating_func(channel) for channel in deriv_image]
    som   = np.sqrt(np.sum(np.power(illum, 2)))
    illum = np.divide(illum, som)

    return illum

Tambien se define *color_constant*, la cual normaliza los colores a travez de los tres canales RGB.

In [3]:
def color_constant(img):
    # Se separa los tres canales
    img_R = img[:,:,0]
    img_G = img[:,:,1]
    img_B = img[:,:,2]    
    
    # Se calcula la media de cada canal y de la imagen
    avg_R = np.mean(img_R)
    avg_G = np.mean(img_G)
    avg_B = np.mean(img_B)
    avg_all = np.mean(img)
    
    # Dado el promedio de cada canal, se calcula su escala correspondiente
    scale_R = (avg_all / avg_R)
    scale_G = (avg_all / avg_G)
    scale_B = (avg_all / avg_B)
    
    # Se ajusta cada canal dado su escala
    img_new = np.zeros(img.shape)
    img_new[:,:,0] = scale_R * img_R  
    img_new[:,:,1] = scale_G * img_G 
    img_new[:,:,2] = scale_B * img_B  
    
    # Se toma el minimo y maximo 
    max_intensity = np.max(np.max(np.max(img_new)))
    min_intensity = np.min(np.min(np.min(img_new)))
    
    # Dado dichos valores, se normaliza la imagen.
    img_normalized = (((img_new - min_intensity) / (max_intensity - min_intensity))*255).astype(np.uint8)

    # Perfil de iluminación (de todas formas este no se usa, pues se estima con la función *grey_edge*) 
    illuminance = [avg_R, avg_G, avg_B]
    
    return img_normalized, illuminance

Para terminar con las funciones auxiliares, se define *correct_image*, la cual corrige la iluminación de la imagen dado una estimación de su perfil (de iluminación).

In [4]:
def correct_image(image, illum):
    """
    Dado el perfil de iluminación estimado, corrige la imagen.
    
    :param image: imagen rgb (NxMx3)
    :param illum: perfil de iluminación estimado
    :return: imagen corregida
    
    Ref: https://github.com/MinaSGorgy/Color-Constancy
    """
    correcting_illum = illum * np.sqrt(3)
    corrected_image = image / 255.
    for channel in range(3):
        corrected_image[:, :, channel] /= correcting_illum[channel]
    return np.clip(corrected_image, 0., 1.)

Con todo ya definido, creamos la función *extract_features*, la cual extrae una variedad de caracteristicas en distintos contextos. Observar que en general se extraen las mismas caracteristicas, variando solamente la imagen de entrada (se prueba extraer luego de distintas correciones o tomando la imagen en RGB o HSV).

Es importante aclarar que hay funciones que estan definidas en este notebook, sino que estan presentes en el script *freatures_funciones.py*. Las mismas realizan funciones menores como calcular los momentos, obtener el LBP, calcular la entropia, etc. Simplemente se separo en otro script para simplificar la lectura del notebook.

In [5]:
def extract_features(image,mask=None):    
    # Se separa en RGB y HSV 
    img_RGB = image
    img_HSV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)
    
    img_masked_RGB = np.ma.array(np.multiply(img_RGB,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask)))
    img_masked_HSV = np.ma.array(np.multiply(img_HSV,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask)))
    

    # Se aplica correción de iluminación y normalización de los canales RGB 
    img_ccRGB,_ = color_constant(image)
    img_ccHSV   = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)
    img_ccRGB = img_ccRGB.astype(np.float64) # Debo cambiar el formato a float64, pues si no lo hago en algunas imagenes tira error.
    img_ccHSV = img_ccHSV.astype(np.float64)
    
        
    img_masked_ccRGB = np.ma.array(np.multiply(img_ccRGB,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask))) 
    img_masked_ccHSV = np.ma.array(np.multiply(img_ccHSV,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask))) 
    
    img_mxRGB = (correct_image(image, grey_edge(image, njet=0, mink_norm=-1, sigma=0))*255).astype(np.uint8)
    img_mxHSV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV) 
    
    img_masked_mxRGB = np.ma.array(np.multiply(img_mxRGB,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask))) 
    img_masked_mxHSV = np.ma.array(np.multiply(img_mxHSV,   np.dstack((mask,mask,mask))), mask=~np.dstack((mask,mask,mask)))
    

    # Momentos 
    img_masked_RGB_CM = img_masked_RGB.astype(np.float64) # Debo cambiar el formato a float64, pues si no lo hago en algunas imagenes tira error.
    img_masked_HSV_CM = img_masked_HSV.astype(np.float64)  

    mean_R, std_R, skew_R, kurt_R, mean_G,  std_G,  skew_G,  kurt_G,  mean_B,  std_B,  skew_B,  kurt_B = momentos(img_masked_RGB_CM)
    mean_H, std_H, skew_H, kurt_H, mean_S,  std_S,  skew_S,  kurt_S,  mean_V,  std_V,  skew_V,  kurt_V = momentos(img_masked_HSV_CM)

    img_masked_ccRGB = img_masked_ccRGB.astype(np.float64) # Debo cambiar el formato a float64, pues si no lo hago en algunas imagenes tira error.
    img_masked_ccHSV = img_masked_ccHSV.astype(np.float64)
    
    mean_ccR, std_ccR, skew_ccR, kurt_ccR, mean_ccG,  std_ccG,  skew_ccG,  kurt_ccG,  mean_ccB,  std_ccB,  skew_ccB,  kurt_ccB = momentos(img_masked_ccRGB)
    mean_ccH, std_ccH, skew_ccH, kurt_ccH, mean_ccS,  std_ccS,  skew_ccS,  kurt_ccS,  mean_ccV,  std_ccV,  skew_ccV,  kurt_ccV = momentos(img_masked_ccHSV)

    img_masked_mxRGB = img_masked_mxRGB.astype(np.float64) # Debo cambiar el formato a float64, pues si no lo hago en algunas imagenes tira error.
    img_masked_mxHSV = img_masked_mxHSV.astype(np.float64)

    mean_mxR, std_mxR, skew_mxR, kurt_mxR, mean_mxG,  std_mxG,  skew_mxG,  kurt_mxG,  mean_mxB,  std_mxB,  skew_mxB,  kurt_mxB = momentos(img_masked_mxRGB)
    mean_mxH, std_mxH, skew_mxH, kurt_mxH, mean_mxS,  std_mxS,  skew_mxS,  kurt_mxS,  mean_mxV,  std_mxV,  skew_mxV,  kurt_mxV = momentos(img_masked_mxHSV)


    # Descriptores de la matriz de coocurrencia 
    GLCM_RGB = descriptores_coocurrencia(img_masked_RGB)
    GLCM_HSV = descriptores_coocurrencia(img_masked_HSV)

    
    # Marcadores 
    Marcador_negro, Marcador_rojo, Marcador_azulg, Marcador_blanco, Marcador_lmarron, Marcador_dmarron = marcadores_melanoma(img_masked_RGB, mask)  
    
    
    # Local Binary Patterns
    lbp_R, lbp_G, lbp_B = LBP(img_masked_RGB)
    lbp_H, lbp_S, lbp_V = LBP(img_masked_HSV)
        
    LBP_CGLF  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V),axis=0)
    
    
    # Descriptores: entropia, uniformidad y suavidad.
    entropyplus_RGB = descriptores_full(img_masked_RGB)
    entropyplus_HSV = descriptores_full(img_masked_HSV)

    
    features = [ mean_R, std_R, skew_R, mean_G,  std_G,  skew_G,  mean_B,  std_B,  skew_B,   
                 mean_H, std_H, skew_H, mean_S,  std_S,  skew_S,  mean_V,  std_V,  skew_V,     
                
                 Marcador_negro, Marcador_blanco, Marcador_lmarron, Marcador_dmarron,
               
                 mean_ccR, std_ccR, skew_ccR, mean_ccG,  std_ccG,  skew_ccG,  mean_ccB,  std_ccB,  skew_ccB, 
                 mean_ccH, std_ccH, skew_ccH, mean_ccS,  std_ccS,  skew_ccS,  mean_ccV,  std_ccV,  skew_ccV, 
               
                 mean_mxR, std_mxR, skew_mxR, mean_mxG,  std_mxG,  skew_mxG,  mean_mxB,  std_mxB,  skew_mxB, 
                 mean_mxH, std_mxH, skew_mxH, mean_mxS,  std_mxS,  skew_mxS,  mean_mxV,  std_mxV,  skew_mxV ]
 
    # Juntamos todo en una unica salida.
    features = np.concatenate((features, GLCM_RGB, GLCM_HSV, LBP_CGLF, entropyplus_RGB, entropyplus_HSV),axis=0)

    return features

### Llamado de la función y creación del csv

Con las funciones ya definidas, basta con recorrer las imágenes en el directorio y extraer las caracteristicas. Para comenzar, se cargan los nombres de las imagenes.

In [11]:
dir_img_train = 'Subconjunto_2018/images/'
dir_seg_train = 'Subconjunto_2018/masks/'

name_csv = pd.read_csv('Subconjunto_2018/list.csv', header=None)
name_list_train = name_csv.iloc[:,0].values

print(name_list_train[:5])
name_csv.head()

['ISIC_0000013' 'ISIC_0000141' 'ISIC_0000160' 'ISIC_0000161'
 'ISIC_0000164']


,0,1
0,ISIC_0000013,1
1,ISIC_0000141,1
2,ISIC_0000160,1
3,ISIC_0000161,1
4,ISIC_0000164,1


Finalmente, extraemos las caracteristicas de cada imagen y las guardamos en un .csv llamado *features_subconjunto_2018.csv*. La corrida de la próxima celda podria demorar unos minutos.

In [13]:
df = pd.DataFrame() # Inicializo el DataFrame

for name in name_list_train: # Recorro todas las imagenes
    img = imread(dir_img_train+name+'.jpg')
    img_seg = imread(dir_seg_train+name+'_Segmentation.png')

    features = extract_features(img,img_seg)
    features = features.reshape(1,features.size)

    descriptores = pd.DataFrame(features)

    df = pd.concat([df,descriptores])
    
data_img = name_csv.copy()

df = df.reset_index(drop=True)
df = pd.concat([data_img,df],axis=1)

# Se le cambia el nombre a las columnas. Si no hacemos esto vamos a tener problemas al evaluar con el modelo.
# Para esto leo otro .csv que tiene las columnas con el nombre correcto.
col_names_df = pd.read_csv('features/features_subconjunto_2018_nuestra_segmentacion_mask.csv') 
df = pd.DataFrame(data=df.values, columns=col_names_df.columns) 

df.to_csv('features_subconjunto_2018.csv', index=False) # Guardo el .csv